# Use Case: Agriculture

Below is an example of merging EMIT CWC scenes and clipping an ECOSTRESS scene to that boundary. I thought for this use case it might be nice to show how to merge the EMIT scenes.

Santa Barbara County has a shapefile [here](https://www.countyofsb.org/1858/Mapping-GIS) with various fields including their crop for 2023 - this might be too complicated to dive into, but could potentially be used to extract data about different crops.

In [ ]:
import os
import glob
import math
import earthaccess
import numpy as np
import pandas as pd
import xarray as xr
from osgeo import gdal
import rasterio as rio
import rioxarray as rxr
from rioxarray.merge import merge_arrays
from matplotlib import pyplot as plt
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import geoviews as gv
import geopandas as gp
import sys
from modules.emit_tools import emit_xarray, ortho_xr

Open ROI bbox and get bounds, need to be in order: left, bottom, right, top.

In [ ]:
# This polygon contains a bounding box around Dangermond and Sedgwick, including the regions between.
bbox = gp.read_file("../data/roi_bbox.geojson").unary_union.bounds
bbox

In [ ]:
ewt_fp1="../../shared/2023-VITALS-Workshop-AGU/data/cropped/ewt/EMIT_L2A_RFL_001_20230401T203751_2309114_002_roi_bbox_cwc.tif"
ewt_fp2="../../shared/2023-VITALS-Workshop-AGU/data/cropped/ewt/EMIT_L2A_RFL_001_20230401T203803_2309114_003_roi_bbox_cwc.tif"
lst_fp = "../../shared/2023-VITALS-Workshop-AGU/data/ECOv002_L2T_LSTE_26860_001_10SGD_20230401T203733_0710_01_LST.tif"

In [ ]:
ewt_ds1 = rxr.open_rasterio(ewt_fp1).squeeze('band',drop=True)
ewt_ds2 = rxr.open_rasterio(ewt_fp2).squeeze('band',drop=True)

In [ ]:
merged = merge_arrays([ewt_ds1,ewt_ds2],bounds=bbox, nodata=np.nan)
merged

In [ ]:
merged.hvplot.image(x='x',y='y',geo=True, cmap='viridis',tiles='ESRI',
                    title=f"{merged.long_name} ({merged.units})",
                 xlabel='Longitude',ylabel='Latitude', frame_width=500,
                 frame_height=500, fontscale=1.5, alpha = 0.7)

Write output

In [ ]:
out_dir = "../../shared/2023-VITALS-Workshop-AGU/data/cropped/dangermond/ewt/"
#out_dir = "../../shared/2023-VITALS-Workshop-AGU/data/cropped/ewt/"
merged_outname=f"{out_dir}{ewt_fp1.split('/')[-1].split('.')[0]}_merged.tif"
merged.rio.to_raster(raster_path=merged_outname, driver='COG')

Open and reproject, regrid, and clip ECOSTRESS LST

In [ ]:
lst_ds = rxr.open_rasterio(lst_fp).squeeze('band',drop=True)

`reproject_match` will reproject the ECOSTRESS scene and regrid to the same as the EMIT CWC scene. We do not have to clip this because our ROI is a box.

In [ ]:
eco_lst_ds_regrid = lst_ds.rio.reproject_match(merged)

In [ ]:
eco_lst_ds_regrid.hvplot.image(x='x',y='y',geo=True, cmap='viridis',tiles='ESRI',
                    title="Regridded 60m ECOSTRESS LST (K)",
                 xlabel='Longitude',ylabel='Latitude', frame_width=500,
                 frame_height=500, fontscale=1.5, alpha = 0.7)

In [ ]:
#Write Output
